In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

/Users/mathiasotnes/anaconda3/envs/autogluon/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Import data
obs_A = pd.read_parquet('../../preprocessing/data/obs_A.parquet')
obs_B = pd.read_parquet('../../preprocessing/data/obs_B.parquet')
obs_C = pd.read_parquet('../../preprocessing/data/obs_C.parquet')
est_A = pd.read_parquet('../../preprocessing/data/est_A.parquet')
est_B = pd.read_parquet('../../preprocessing/data/est_B.parquet')
est_C = pd.read_parquet('../../preprocessing/data/est_C.parquet')
test_A = pd.read_parquet('../../preprocessing/data/test_A.parquet')
test_B = pd.read_parquet('../../preprocessing/data/test_B.parquet')
test_C = pd.read_parquet('../../preprocessing/data/test_C.parquet')

In [15]:
# Concatinate
A = pd.concat([obs_A, est_A])
B = pd.concat([obs_B, est_B])
C = pd.concat([obs_C, est_C])

# Data splits for submissions
X_A = A.drop(columns='pv_measurement')
y_A = A['pv_measurement']
X_B = B.drop(columns='pv_measurement')
y_B = B['pv_measurement']
X_C = C.drop(columns='pv_measurement')
y_C = C['pv_measurement']

# Data splits for testing
train_A, test_A = train_test_split(A, test_size=0.2, shuffle=True, random_state=42)
X_train_A = train_A.drop(columns='pv_measurement')
y_train_A = train_A['pv_measurement']
X_test_A = test_A.drop(columns='pv_measurement')
y_test_A = test_A['pv_measurement']

train_B, test_B = train_test_split(B, test_size=0.2, shuffle=True, random_state=42)
X_train_B = train_B.drop(columns='pv_measurement')
y_train_B = train_B['pv_measurement']
X_test_B = test_B.drop(columns='pv_measurement')
y_test_B = test_B['pv_measurement']

train_C, test_C = train_test_split(C, test_size=0.2, shuffle=True, random_state=42)
X_train_C = train_C.drop(columns='pv_measurement')
y_train_C = train_C['pv_measurement']
X_test_C = test_C.drop(columns='pv_measurement')
y_test_C = test_C['pv_measurement']



In [5]:
# Optional AutoGluon preprocessing
# train_data = TabularDataset(obs_A)
# test_data = TabularDataset(est_A)



In [6]:
label = 'pv_measurement'
test_A.describe()

,snow_accumulation,total_radiation,sfc_pressure:hPa,month,year,date_forecast_fft_amplitude,date_forecast_fft_phase,sun_elevation:d_fft_amplitude,sun_elevation:d_fft_phase,clear_sky_rad:W_rate_of_change,...,pressure_gradient,temp_dewpoint_diff,sun_elevation:d_rolling_avg_6,super_cooled_liquid_water:kgm2,relative_humidity_1000hPa:p,is_day:idx,is_in_shadow:idx,pressure_50m:hPa,sun_azimuth:d,absolute_humidity_2m:gm3
count,720.000000,720.000000,720.000000,720.000000,720.0,7.200000e+02,720.000000,720.000000,720.000000,720.000000,...,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,0.001563,199.684021,1015.920654,5.666667,2023.0,1.697683e+18,-0.004363,65.334998,-0.004363,0.000000,...,-6.041284,3.923195,20.299151,0.063924,70.750069,0.793750,0.225347,1009.852600,179.181381,8.205903
std,0.029809,215.015656,9.817863,0.596699,0.0,4.523976e+19,2.399267,736.599572,2.397744,83.207748,...,0.143694,3.379657,16.830749,0.108792,15.531467,0.385538,0.405328,9.717662,103.971336,2.178425
min,0.000000,0.000000,985.450012,5.000000,2023.0,1.296000e+15,-3.141593,0.947522,-3.141593,-131.675018,...,-6.349976,-0.900024,-10.541000,0.000000,24.824999,0.000000,0.000000,979.625000,13.441751,3.200000
25%,0.000000,9.068750,1010.487503,5.000000,2023.0,3.072918e+15,-2.356614,4.725976,-2.379941,-61.381248,...,-6.150024,1.443741,3.533635,0.000000,60.368752,1.000000,0.000000,1004.437515,89.981750,6.693750
50%,0.000000,121.874996,1019.662506,6.000000,2023.0,3.949678e+15,-0.718200,5.980017,-0.283418,0.000000,...,-6.075012,3.100006,18.384875,0.000000,73.550003,1.000000,0.000000,1013.562500,180.960999,8.050000
75%,0.000000,343.337494,1023.031265,6.000000,2023.0,7.183007e+15,2.354935,11.549801,2.377725,65.218752,...,-5.925049,5.531242,36.121531,0.100000,83.275002,1.000000,0.000000,1017.000000,271.244804,9.981250
max,0.625000,764.924988,1028.699951,7.000000,2023.0,1.213921e+21,3.139914,14611.998002,3.141193,132.500000,...,-5.650024,18.174988,46.352375,0.600000,97.000000,1.000000,1.000000,1022.500000,348.118500,13.675000


In [7]:
# Fit the predictors
predictor_A = TabularPredictor(label=label, eval_metric='mean_absolute_error').fit(A, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20231107_064802/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231107_064802/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 21.6.0: Wed Aug 10 14:28:23 PDT 2022; root:xnu-8020.141.5~2/RELEASE_ARM64_T6000
Disk Space Avail:   96.38 GB / 494.38 GB (19.5%)
Train Data Rows:    34042
Train Data Columns: 60
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 631.39124, 1166.42299)
	If 'regression' is not the correct problem_type, please manually specify the problem_type paramet

[1000]	valid_set's l1: 170.43
[2000]	valid_set's l1: 163.453
[3000]	valid_set's l1: 160.225
[4000]	valid_set's l1: 157.905
[5000]	valid_set's l1: 156.506
[6000]	valid_set's l1: 155.533
[7000]	valid_set's l1: 154.87
[8000]	valid_set's l1: 154.453
[9000]	valid_set's l1: 154.024
[10000]	valid_set's l1: 153.716
[1000]	valid_set's l1: 163.737
[2000]	valid_set's l1: 156.9
[3000]	valid_set's l1: 153.683
[4000]	valid_set's l1: 151.58
[5000]	valid_set's l1: 150.207
[6000]	valid_set's l1: 149.435
[7000]	valid_set's l1: 148.756
[8000]	valid_set's l1: 148.236
[9000]	valid_set's l1: 147.867
[10000]	valid_set's l1: 147.586
[1000]	valid_set's l1: 164.757
[2000]	valid_set's l1: 157.931
[3000]	valid_set's l1: 154.548
[4000]	valid_set's l1: 152.275
[5000]	valid_set's l1: 151.093
[6000]	valid_set's l1: 150.237
[7000]	valid_set's l1: 149.593
[8000]	valid_set's l1: 149.128
[9000]	valid_set's l1: 148.743
[10000]	valid_set's l1: 148.469
[1000]	valid_set's l1: 161.895
[2000]	valid_set's l1: 155.479
[3000]	val

	-147.5197	 = Validation score   (-mean_absolute_error)
	4392.25s	 = Training   runtime
	4.01s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 167.73
[2000]	valid_set's l1: 163.161
[3000]	valid_set's l1: 161.531
[4000]	valid_set's l1: 160.763
[5000]	valid_set's l1: 160.306
[6000]	valid_set's l1: 160.065
[7000]	valid_set's l1: 159.914
[8000]	valid_set's l1: 159.818
[9000]	valid_set's l1: 159.777
[10000]	valid_set's l1: 159.756
[1000]	valid_set's l1: 161.773
[2000]	valid_set's l1: 157.2
[3000]	valid_set's l1: 155.478
[4000]	valid_set's l1: 154.673
[5000]	valid_set's l1: 154.361
[6000]	valid_set's l1: 154.187
[7000]	valid_set's l1: 154.049
[8000]	valid_set's l1: 153.98
[9000]	valid_set's l1: 153.912
[10000]	valid_set's l1: 153.876
[1000]	valid_set's l1: 164.272
[2000]	valid_set's l1: 159.782
[3000]	valid_set's l1: 158.069
[4000]	valid_set's l1: 157.441
[5000]	valid_set's l1: 157.025
[6000]	valid_set's l1: 156.802
[7000]	valid_set's l1: 156.644
[8000]	valid_set's l1: 156.547
[9000]	valid_set's l1: 156.503
[10000]	valid_set's l1: 156.458
[1000]	valid_set's l1: 159.933
[2000]	valid_set's l1: 155.948
[3000]	va

	-155.6693	 = Validation score   (-mean_absolute_error)
	6707.72s	 = Training   runtime
	4.17s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-176.126	 = Validation score   (-mean_absolute_error)
	47.25s	 = Training   runtime
	0.86s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-160.1051	 = Validation score   (-mean_absolute_error)
	810.19s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-177.8953	 = Validation score   (-mean_absolute_error)
	7.02s	 = Training   runtime
	0.83s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-188.0007	 = Validation score   (-mean_absolute_error)
	123.78s	 = Training   runtime
	0.17s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with S

[1000]	valid_set's l1: 161.667
[2000]	valid_set's l1: 159.991
[3000]	valid_set's l1: 159.617
[4000]	valid_set's l1: 159.511
[5000]	valid_set's l1: 159.476
[6000]	valid_set's l1: 159.466
[7000]	valid_set's l1: 159.462
[8000]	valid_set's l1: 159.46
[9000]	valid_set's l1: 159.459
[10000]	valid_set's l1: 159.459
[1000]	valid_set's l1: 151.975
[2000]	valid_set's l1: 150.453
[3000]	valid_set's l1: 150.097
[4000]	valid_set's l1: 149.99
[5000]	valid_set's l1: 149.954
[6000]	valid_set's l1: 149.94
[7000]	valid_set's l1: 149.934
[8000]	valid_set's l1: 149.932
[9000]	valid_set's l1: 149.93
[10000]	valid_set's l1: 149.93
[1000]	valid_set's l1: 158.469
[2000]	valid_set's l1: 156.63
[3000]	valid_set's l1: 156.265
[4000]	valid_set's l1: 156.157
[5000]	valid_set's l1: 156.123
[6000]	valid_set's l1: 156.111
[7000]	valid_set's l1: 156.106
[8000]	valid_set's l1: 156.104
[9000]	valid_set's l1: 156.103
[10000]	valid_set's l1: 156.103
[1000]	valid_set's l1: 151.27
[2000]	valid_set's l1: 149.763
[3000]	valid

	-153.0901	 = Validation score   (-mean_absolute_error)
	5977.98s	 = Training   runtime
	11.24s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-144.8104	 = Validation score   (-mean_absolute_error)
	0.24s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting 9 L2 models ...
Fitting model: LightGBMXT_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-135.6788	 = Validation score   (-mean_absolute_error)
	38.22s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-133.2799	 = Validation score   (-mean_absolute_error)
	26.49s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ...
	-132.4075	 = Validation score   (-mean_absolute_error)
	78.84s	 = Training   runtime
	0.88s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8

In [8]:
predictor_B = TabularPredictor(label=label, eval_metric='mean_absolute_error').fit(B, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20231107_122119/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231107_122119/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 21.6.0: Wed Aug 10 14:28:23 PDT 2022; root:xnu-8020.141.5~2/RELEASE_ARM64_T6000
Disk Space Avail:   92.23 GB / 494.38 GB (18.7%)
Train Data Rows:    25924
Train Data Columns: 60
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1152.3, -0.0, 107.07661, 212.51086)
	If 'regression' is not the correct problem_type, please manually specify the problem_type paramete

[1000]	valid_set's l1: 23.3336
[2000]	valid_set's l1: 22.4602
[3000]	valid_set's l1: 22.1751
[4000]	valid_set's l1: 21.9769
[5000]	valid_set's l1: 21.8515
[6000]	valid_set's l1: 21.7782
[7000]	valid_set's l1: 21.7165
[8000]	valid_set's l1: 21.6743
[9000]	valid_set's l1: 21.6473
[10000]	valid_set's l1: 21.6171
[1000]	valid_set's l1: 22.4409
[2000]	valid_set's l1: 21.6327
[3000]	valid_set's l1: 21.2415
[4000]	valid_set's l1: 21.0214
[5000]	valid_set's l1: 20.8951
[6000]	valid_set's l1: 20.811
[7000]	valid_set's l1: 20.753
[8000]	valid_set's l1: 20.7211
[9000]	valid_set's l1: 20.6926
[10000]	valid_set's l1: 20.6671
[1000]	valid_set's l1: 21.749
[2000]	valid_set's l1: 20.8961
[3000]	valid_set's l1: 20.595
[4000]	valid_set's l1: 20.3872
[5000]	valid_set's l1: 20.2699
[6000]	valid_set's l1: 20.2036
[7000]	valid_set's l1: 20.1627
[8000]	valid_set's l1: 20.1325
[9000]	valid_set's l1: 20.1112
[10000]	valid_set's l1: 20.0923
[1000]	valid_set's l1: 23.5298
[2000]	valid_set's l1: 22.737
[3000]	val

	-20.9433	 = Validation score   (-mean_absolute_error)
	669.39s	 = Training   runtime
	4.73s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 23.6776
[2000]	valid_set's l1: 23.211
[3000]	valid_set's l1: 23.0919
[4000]	valid_set's l1: 23.0388
[5000]	valid_set's l1: 23.0222
[6000]	valid_set's l1: 23.0075
[7000]	valid_set's l1: 22.999
[8000]	valid_set's l1: 22.9971
[9000]	valid_set's l1: 22.9945
[10000]	valid_set's l1: 22.9936
[1000]	valid_set's l1: 22.3541
[2000]	valid_set's l1: 21.9036
[3000]	valid_set's l1: 21.7672
[4000]	valid_set's l1: 21.6933
[5000]	valid_set's l1: 21.6604
[6000]	valid_set's l1: 21.6382
[7000]	valid_set's l1: 21.6271
[8000]	valid_set's l1: 21.6218
[9000]	valid_set's l1: 21.6175
[10000]	valid_set's l1: 21.6147
[1000]	valid_set's l1: 21.4606
[2000]	valid_set's l1: 21.0062
[3000]	valid_set's l1: 20.8521
[4000]	valid_set's l1: 20.8084
[5000]	valid_set's l1: 20.7756
[6000]	valid_set's l1: 20.7638
[7000]	valid_set's l1: 20.7547
[8000]	valid_set's l1: 20.7506
[9000]	valid_set's l1: 20.7486
[10000]	valid_set's l1: 20.7483
[1000]	valid_set's l1: 23.2671
[2000]	valid_set's l1: 22.9285
[3000]	

	-21.9182	 = Validation score   (-mean_absolute_error)
	714.49s	 = Training   runtime
	4.96s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-25.3946	 = Validation score   (-mean_absolute_error)
	63.22s	 = Training   runtime
	0.58s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-22.486	 = Validation score   (-mean_absolute_error)
	2616.9s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-25.721	 = Validation score   (-mean_absolute_error)
	5.04s	 = Training   runtime
	0.54s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-26.9379	 = Validation score   (-mean_absolute_error)
	91.56s	 = Training   runtime
	0.12s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with Sequentia

[1000]	valid_set's l1: 22.7453
[2000]	valid_set's l1: 22.6369
[3000]	valid_set's l1: 22.618
[4000]	valid_set's l1: 22.6133
[5000]	valid_set's l1: 22.6121
[6000]	valid_set's l1: 22.6117
[7000]	valid_set's l1: 22.6115
[8000]	valid_set's l1: 22.6115
[9000]	valid_set's l1: 22.6115
[10000]	valid_set's l1: 22.6115
[1000]	valid_set's l1: 21.8937
[2000]	valid_set's l1: 21.7462
[3000]	valid_set's l1: 21.7213
[4000]	valid_set's l1: 21.7148
[5000]	valid_set's l1: 21.713
[6000]	valid_set's l1: 21.7125
[7000]	valid_set's l1: 21.7124
[8000]	valid_set's l1: 21.7123
[9000]	valid_set's l1: 21.7123
[10000]	valid_set's l1: 21.7123
[1000]	valid_set's l1: 20.8647
[2000]	valid_set's l1: 20.7256
[3000]	valid_set's l1: 20.7033
[4000]	valid_set's l1: 20.6975
[5000]	valid_set's l1: 20.6962
[6000]	valid_set's l1: 20.6957
[7000]	valid_set's l1: 20.6955
[8000]	valid_set's l1: 20.6954
[9000]	valid_set's l1: 20.6954
[10000]	valid_set's l1: 20.6954
[1000]	valid_set's l1: 22.6967
[2000]	valid_set's l1: 22.5918
[3000]	

	-21.8831	 = Validation score   (-mean_absolute_error)
	3543.6s	 = Training   runtime
	9.62s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-20.5565	 = Validation score   (-mean_absolute_error)
	0.18s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting 9 L2 models ...
Fitting model: LightGBMXT_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-20.3457	 = Validation score   (-mean_absolute_error)
	36.52s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-20.2678	 = Validation score   (-mean_absolute_error)
	25.27s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ...
	-19.7702	 = Validation score   (-mean_absolute_error)
	63.21s	 = Training   runtime
	0.55s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fit

In [9]:
predictor_C = TabularPredictor(label=label, eval_metric='mean_absolute_error').fit(C, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20231107_145557/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231107_145557/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 21.6.0: Wed Aug 10 14:28:23 PDT 2022; root:xnu-8020.141.5~2/RELEASE_ARM64_T6000
Disk Space Avail:   89.15 GB / 494.38 GB (18.0%)
Train Data Rows:    21135
Train Data Columns: 60
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (999.6, -0.0, 95.84966, 179.44873)
	If 'regression' is not the correct problem_type, please manually specify the problem_type param

[1000]	valid_set's l1: 21.6196
[2000]	valid_set's l1: 20.842
[3000]	valid_set's l1: 20.5086
[4000]	valid_set's l1: 20.3225
[5000]	valid_set's l1: 20.2335
[6000]	valid_set's l1: 20.1701
[7000]	valid_set's l1: 20.1233
[8000]	valid_set's l1: 20.0988
[9000]	valid_set's l1: 20.0779
[10000]	valid_set's l1: 20.0637
[1000]	valid_set's l1: 19.3213
[2000]	valid_set's l1: 18.7425
[3000]	valid_set's l1: 18.5101
[4000]	valid_set's l1: 18.3965
[5000]	valid_set's l1: 18.3305
[6000]	valid_set's l1: 18.2937
[7000]	valid_set's l1: 18.2685
[8000]	valid_set's l1: 18.2468
[9000]	valid_set's l1: 18.2264
[10000]	valid_set's l1: 18.2111
[1000]	valid_set's l1: 21.2692
[2000]	valid_set's l1: 20.4659
[3000]	valid_set's l1: 20.126
[4000]	valid_set's l1: 19.9805
[5000]	valid_set's l1: 19.9199
[6000]	valid_set's l1: 19.8785
[7000]	valid_set's l1: 19.8403
[8000]	valid_set's l1: 19.8283
[9000]	valid_set's l1: 19.8165
[10000]	valid_set's l1: 19.8115
[1000]	valid_set's l1: 20.4388
[2000]	valid_set's l1: 19.7168
[3000]	

	-18.9231	 = Validation score   (-mean_absolute_error)
	665.07s	 = Training   runtime
	3.93s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 21.7416
[2000]	valid_set's l1: 21.3606
[3000]	valid_set's l1: 21.2381
[4000]	valid_set's l1: 21.1978
[5000]	valid_set's l1: 21.1797
[6000]	valid_set's l1: 21.1697
[7000]	valid_set's l1: 21.1639
[8000]	valid_set's l1: 21.1615
[9000]	valid_set's l1: 21.1593
[10000]	valid_set's l1: 21.158
[1000]	valid_set's l1: 19.618
[2000]	valid_set's l1: 19.3509
[3000]	valid_set's l1: 19.2768
[4000]	valid_set's l1: 19.2351
[5000]	valid_set's l1: 19.2103
[6000]	valid_set's l1: 19.1989
[7000]	valid_set's l1: 19.1969
[8000]	valid_set's l1: 19.1947
[9000]	valid_set's l1: 19.1929
[10000]	valid_set's l1: 19.1912
[1000]	valid_set's l1: 21.2936
[2000]	valid_set's l1: 20.9831
[3000]	valid_set's l1: 20.943
[4000]	valid_set's l1: 20.9232
[5000]	valid_set's l1: 20.9098
[6000]	valid_set's l1: 20.9012
[7000]	valid_set's l1: 20.899
[8000]	valid_set's l1: 20.8971
[9000]	valid_set's l1: 20.8958
[10000]	valid_set's l1: 20.8951
[1000]	valid_set's l1: 21.0861
[2000]	valid_set's l1: 20.6821
[3000]	va

	-19.8366	 = Validation score   (-mean_absolute_error)
	705.28s	 = Training   runtime
	4.31s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ...
	-23.0132	 = Validation score   (-mean_absolute_error)
	39.48s	 = Training   runtime
	0.45s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-20.1425	 = Validation score   (-mean_absolute_error)
	808.75s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L1 ...
	-23.1162	 = Validation score   (-mean_absolute_error)
	6.68s	 = Training   runtime
	0.48s	 = Validation runtime
Fitting model: NeuralNetFastAI_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-24.3509	 = Validation score   (-mean_absolute_error)
	781.57s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: XGBoost_BAG_L1 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with Sequent

[1000]	valid_set's l1: 21.4038
[2000]	valid_set's l1: 21.3123
[3000]	valid_set's l1: 21.2991
[4000]	valid_set's l1: 21.2968
[5000]	valid_set's l1: 21.2964
[6000]	valid_set's l1: 21.2962
[7000]	valid_set's l1: 21.2962
[8000]	valid_set's l1: 21.2962
[9000]	valid_set's l1: 21.2962
[10000]	valid_set's l1: 21.2962
[1000]	valid_set's l1: 19.4308
[2000]	valid_set's l1: 19.3609
[3000]	valid_set's l1: 19.3477
[4000]	valid_set's l1: 19.3452
[5000]	valid_set's l1: 19.3449
[6000]	valid_set's l1: 19.3447
[7000]	valid_set's l1: 19.3447
[8000]	valid_set's l1: 19.3447
[9000]	valid_set's l1: 19.3447
[10000]	valid_set's l1: 19.3447
[1000]	valid_set's l1: 20.9361
[2000]	valid_set's l1: 20.864
[3000]	valid_set's l1: 20.8513
[4000]	valid_set's l1: 20.8488
[5000]	valid_set's l1: 20.8482
[6000]	valid_set's l1: 20.848
[7000]	valid_set's l1: 20.848
[8000]	valid_set's l1: 20.848
[9000]	valid_set's l1: 20.848
[10000]	valid_set's l1: 20.848
[1000]	valid_set's l1: 20.6958
[2000]	valid_set's l1: 20.6119
[3000]	vali

	-20.0488	 = Validation score   (-mean_absolute_error)
	3452.93s	 = Training   runtime
	17.98s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-18.6541	 = Validation score   (-mean_absolute_error)
	0.16s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting 9 L2 models ...
Fitting model: LightGBMXT_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-18.4836	 = Validation score   (-mean_absolute_error)
	42.46s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-18.5029	 = Validation score   (-mean_absolute_error)
	32.06s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ...
	-17.9449	 = Validation score   (-mean_absolute_error)
	32.43s	 = Training   runtime
	0.46s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | F

In [10]:
# predictor_A.evaluate(test_A, silent=True)
# predictor_B.evaluate(B, silent=True)
# predictor_C.evaluate(C, silent=True)

In [12]:
# predictor_A.leaderboard(test_A, silent=False)
# predictor_B.leaderboard(B, silent=True)
# predictor_C.leaderboard(C, silent=True)

predictor_A.plot_ensemble_model()
predictor_B.plot_ensemble_model()
predictor_C.plot_ensemble_model()

'AutogluonModels/ag-20231107_145557/ensemble_model.png'

In [ ]:
# Loading best predictors
# predictor_A = TabularPredictor.load("AutogluonModels/ag-20231106_113214/")
# predictor_B = TabularPredictor.load("AutogluonModels/ag-20231106_114820/")
# predictor_C = TabularPredictor.load("AutogluonModels/ag-20231106_120433/")

In [13]:
test_A = TabularDataset(test_A)
test_B = TabularDataset(test_B)
test_C = TabularDataset(test_C)

test_A.describe()

,snow_accumulation,total_radiation,sfc_pressure:hPa,month,year,date_forecast_fft_amplitude,date_forecast_fft_phase,sun_elevation:d_fft_amplitude,sun_elevation:d_fft_phase,clear_sky_rad:W_rate_of_change,...,pressure_gradient,temp_dewpoint_diff,sun_elevation:d_rolling_avg_6,super_cooled_liquid_water:kgm2,relative_humidity_1000hPa:p,is_day:idx,is_in_shadow:idx,pressure_50m:hPa,sun_azimuth:d,absolute_humidity_2m:gm3
count,720.000000,720.000000,720.000000,720.000000,720.0,7.200000e+02,720.000000,720.000000,720.000000,720.000000,...,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000
mean,0.001563,199.684021,1015.920654,5.666667,2023.0,1.697683e+18,-0.004363,65.334998,-0.004363,0.000000,...,-6.041284,3.923195,20.299151,0.063924,70.750069,0.793750,0.225347,1009.852600,179.181381,8.205903
std,0.029809,215.015656,9.817863,0.596699,0.0,4.523976e+19,2.399267,736.599572,2.397744,83.207748,...,0.143694,3.379657,16.830749,0.108792,15.531467,0.385538,0.405328,9.717662,103.971336,2.178425
min,0.000000,0.000000,985.450012,5.000000,2023.0,1.296000e+15,-3.141593,0.947522,-3.141593,-131.675018,...,-6.349976,-0.900024,-10.541000,0.000000,24.824999,0.000000,0.000000,979.625000,13.441751,3.200000
25%,0.000000,9.068750,1010.487503,5.000000,2023.0,3.072918e+15,-2.356614,4.725976,-2.379941,-61.381248,...,-6.150024,1.443741,3.533635,0.000000,60.368752,1.000000,0.000000,1004.437515,89.981750,6.693750
50%,0.000000,121.874996,1019.662506,6.000000,2023.0,3.949678e+15,-0.718200,5.980017,-0.283418,0.000000,...,-6.075012,3.100006,18.384875,0.000000,73.550003,1.000000,0.000000,1013.562500,180.960999,8.050000
75%,0.000000,343.337494,1023.031265,6.000000,2023.0,7.183007e+15,2.354935,11.549801,2.377725,65.218752,...,-5.925049,5.531242,36.121531,0.100000,83.275002,1.000000,0.000000,1017.000000,271.244804,9.981250
max,0.625000,764.924988,1028.699951,7.000000,2023.0,1.213921e+21,3.139914,14611.998002,3.141193,132.500000,...,-5.650024,18.174988,46.352375,0.600000,97.000000,1.000000,1.000000,1022.500000,348.118500,13.675000


In [14]:
# Create submission

output_file = 'autogluon_submission.csv'

pred_A = predictor_A.predict(test_A)
pred_B = predictor_B.predict(test_B)
pred_C = predictor_C.predict(test_C)

pred_A = np.clip(pred_A, 0, None)
pred_B = np.clip(pred_B, 0, None)
pred_C = np.clip(pred_C, 0, None)

# Concatenate predictions
predictions = np.concatenate([pred_A, pred_B, pred_C])

# Create an id array
ids = np.arange(0, len(predictions))

# Create a DataFrame
df = pd.DataFrame({
    'id': ids,
    'prediction': predictions
})

# Save to CSV
df.to_csv(output_file, index=False)
print(f"Submission saved to {output_file}")

Submission saved to autogluon_submission.csv


In [17]:
# For testing
pred_A = predictor_A.predict(X_A)
pred_B = predictor_B.predict(X_B)
pred_C = predictor_C.predict(X_C)

# Evaluate 
mae_A = mean_absolute_error(y_A, pred_A)
mae_B = mean_absolute_error(y_B, pred_B)
mae_C = mean_absolute_error(y_C, pred_C)

print(f"MAE A: {mae_A}")
print(f"MAE B: {mae_B}")
print(f"MAE C: {mae_C}")